<a href="https://colab.research.google.com/github/nchaudh03/AML-HCDR/blob/master/DS%20PROJ-%20Personal%20Version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion, _fit_transform_one, _transform_one
from sklearn.preprocessing import OneHotEncoder, Imputer,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin
from sklearn.externals.joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy import sparse
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier



#os.chdir(r'C:\Users\naimesh.chaudhari\Downloads')
#df  = pd.read_csv('previous_application.csv')

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def Setupkaggle():
  #Kaggle Setup for Google Drive
  !mkdir .kaggle
  with open(r'/content/drive/My Drive/kaggle.json') as file:
    data = json.load(file)
  with open('/content/.kaggle/kaggle.json', 'w') as file:
      json.dump(data, file)
  !chmod 600 /content/.kaggle/kaggle.json
  !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
class ColumnExtractor(TransformerMixin):
    def __init__(self, typ = "num",group = "" ):
        self.typ = typ
        self.group = group
    def fit(self, X, y=None):
        return self
    def transform (self,X):
        cat  = []
        num = []
        Xs  = X
        for col in X.columns:
            if Xs[col].dtypes == 'object':
                cat.append(col)
            else:
                num.append(col)
        if self.group != "":
            if self.group in num:
                cat.append(self.group)
            else:
                num.append(self.group)
        if self.typ == 'num':
            final = Xs[num]
        else:
            final = Xs[cat]
        return final
    

In [0]:
class NumericAggeregator(TransformerMixin):
    def __init__(self,group="",drop=""):
        self.group = group
        self.drop = drop
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        Xs = X
        for tst in self.drop:
          if tst in Xs.columns:
            Xs = Xs.drop(tst,axis = 1)
        Xs = Xs.groupby(self.group).agg([np.sum, np.mean,max,min,count])  
        Xs.columns = Xs.columns.map('_'.join)
        return Xs
   

In [0]:
#might need an encoder class before an aggeregator class
class CatAggeregator(TransformerMixin):
    def __init__(self, group="",drop=""):
        self.group = group
        self.drop = drop
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        Xs  = X
        for tst in self.drop:
          if tst in Xs.columns:
            Xs = Xs.drop(tst,axis = 1)
        Xs = Xs.groupby(self.group).agg([np.mean])  
        Xs.columns = Xs.columns.map('_'.join)
        return Xs

In [0]:

class PandasFeatureUnion(FeatureUnion):
    def fit_transform(self, X, y=None, **fit_params):
        self._validate_transformers()
        result = Parallel(n_jobs=self.n_jobs)(
            delayed(_fit_transform_one)(
                transformer=trans,
                X=X,
                y=y,
                weight=weight,
                **fit_params)
            for name, trans, weight in self._iter())

        if not result:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        Xs, transformers = zip(*result)
        self._update_transformer_list(transformers)
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

    def merge_dataframes_by_column(self, Xs):
        return pd.concat(Xs, axis="columns", copy=False)

    def transform(self, X):
        Xs = Parallel(n_jobs=self.n_jobs)(
            delayed(_transform_one)(
                transformer=trans,
                X=X,
                y=None,
                weight=weight)
            for name, trans, weight in self._iter())
        if not Xs:
            # All transformers are None
            return np.zeros((X.shape[0], 0))
        if any(sparse.issparse(f) for f in Xs):
            Xs = sparse.hstack(Xs).tocsr()
        else:
            Xs = self.merge_dataframes_by_column(Xs)
        return Xs

In [0]:
class dfSimpleImputer(TransformerMixin):
    def __init__(self, missing_values=np.nan,strategy='constant'):
        self.missing_values = missing_values
        self.strategy  = strategy
    def fit(self, X):
        self.imp = SimpleImputer(missing_values=np.nan, strategy='constant').fit(X)
        return self
    def transform(self , X):
        dat = self.imp.transform(X)
        final = pd.DataFrame(dat, columns = X.columns)
        return final
    def fit_transform(self, X, y= None):
        self.imp = SimpleImputer(missing_values=np.nan, strategy='constant').fit(X)
        dat = self.imp.transform(X)
        final = pd.DataFrame(dat, columns = X.columns)
        return final
        

In [0]:
class dfOneHotEncoder(TransformerMixin):
    def __init__(self, key = "", ind = ""):
        self.key = key
        self.ind = ind
    def fit(self, X):
        if self.key != "":
          self.ind = X[self.key]
          X.drop(self.key, inplace = True, axis = 1)
        self.imp = OneHotEncoder(handle_unknown = 'ignore').fit(X)
        return self
    
    def transform(self , X):
        if self.key != "":
          self.ind = X[self.key]
          X.drop(self.key, inplace = True, axis = 1)
        dat = self.imp.transform(X).toarray()
        final = pd.DataFrame(dat, columns = self.imp.get_feature_names())
        if self.key != "":
          final[self.key] = self.ind
        return final
    
    def fit_transform(self, X, y= None ):
        if self.key != "":
          self.ind = X[self.key]
          X.drop(self.key, inplace = True, axis = 1)
        self.imp = OneHotEncoder(handle_unknown = 'ignore').fit(X)
        dat = self.imp.transform(X).toarray()
        final = pd.DataFrame(dat, columns = self.imp.get_feature_names())
        if self.key != "":
          final[self.key] = self.ind
        return final
        

In [0]:
class dfStandardScaler(TransformerMixin):
    def fit(self, X, y=None):
        self.imp = StandardScaler().fit(X)
        return self
    def transform(self, X):
        dat = self.imp.transform(X)
        final = pd.DataFrame(dat, columns = X.columns)
        return final
    def fit_transform(self, X, y=None):
        self.imp = StandardScaler().fit(X)
        dat = self.imp.transform(X)
        final = pd.DataFrame(dat, columns = X.columns, index = X.index)
        return final

In [0]:
def Agg_Pipe(filename = "", grp = "", drp = ""):
  fl = pd.read_csv(r'/content/drive/My Drive/HCDR Project/' + filename)
  Npipe = Pipeline([('cl,', ColumnExtractor(typ  = 'num',group = grp)),
                          ('imp',dfSimpleImputer(missing_values=np.nan, strategy='constant')),
                          ('agg', NumericAggeregator(group = grp,drop = drp)),
                          ('scl',dfStandardScaler())])
  
  
  if filename != 'installments_payments.csv':
    
    Cpipe = Pipeline([('cl,', ColumnExtractor(typ  = 'cat', group =grp )),
                            ('imp',dfSimpleImputer(missing_values=np.nan, strategy='constant')),
                            ('encode', dfOneHotEncoder(key =grp )),
                            ('agg', CatAggeregator(group = grp, drop = drp))])
    combined_features = PandasFeatureUnion([("num", Npipe),
                                                 ("cat", Cpipe)])

    data = combined_features.fit_transform(fl)
    data[grp] = data.index
  
  
  else :
    combined_features = PandasFeatureUnion([("num", Npipe)])
    data = combined_features.fit_transform(fl)
    data[grp] = data.index
  
  
  return data





In [0]:
def combineSuppoutFiles():
  pos = Agg_Pipe('POS_CASH_balance.csv',grp = 'SK_ID_CURR',drp = ['SK_ID_PREV'])
  Pv = Agg_Pipe('previous_application.csv',grp = 'SK_ID_CURR',drp = ['SK_ID_PREV'])
  cc = Agg_Pipe('credit_card_balance.csv',grp= 'SK_ID_CURR',drp= ['SK_ID_PREV'])
  ip = Agg_Pipe('installments_payments.csv',grp= 'SK_ID_CURR',drp = ['SK_ID_PREV'])
  bu = Agg_Pipe('bureau.csv',grp = 'SK_ID_CURR',drp= ['SK_ID_BUREAU','SK_ID_PREV'])

  ctr = 1
  filetojoin = [pos ,Pv,cc,ip,bu]
  for f in filetojoin:
    if ctr == 1:
      final = pos
    else:
      final = final.merge(f, how = 'left', on = 'SK_ID_CURR')
    ctr += 1


  final.fillna(0,inplace = True)
  return final

In [0]:
def submission(model , final, combined_features):
  #Sub pipeline
    test = pd.read_csv(r'/content/drive/My Drive/HCDR Project/application_test.csv')
    test_ids = test['SK_ID_CURR']
    test.drop(['SK_ID_CURR'], inplace = True, axis = 1)
    tst = combined_features.transform(test)
    tst['SK_ID_CURR'] = test_ids
    #merging final
    tst = tst.merge(final, how  = 'left', on = 'SK_ID_CURR')
    tst.drop(['SK_ID_CURR'], axis = 1, inplace = True)
    tst.fillna(0, inplace = True)
    pred = model.predict_proba(tst)[:,1]
    sub = pd.DataFrame(test_ids)
    sub['TARGET'] = pred
    sub.columns = ['SK_ID_CURR','TARGET']
    sub.to_csv('submission.csv', index = False)

  
#submission(model, final,combined_features )
#!kaggle competitions submit -c home-credit-default-risk -f submission.csv -m "Logistic Weighted"
#!kaggle competitions submissions home-credit-default-risk  



def roc(pred, y):
  fpr, tpr,_=roc_curve(pred,y,drop_intermediate=False)
  plt.figure()
  plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve')
  plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
  plt.xlabel('FPR')
  plt.ylabel('TPR')
  plt.title('ROC curve')
  plt.show()
  roc_auc_score(pred,y)
  

In [0]:
#train pipeline

Npipe_prevapp = Pipeline([('cl,', ColumnExtractor(typ  = 'num',group ='')),
                          ('imp',dfSimpleImputer(missing_values=np.nan, strategy='constant')),
                          ('scl',dfStandardScaler())])



Cpipe_prevapp = Pipeline([('cl,', ColumnExtractor(typ  = 'cat', group ='' )),
                          ('imp',dfSimpleImputer(missing_values=np.nan, strategy='constant')),
                          ('encode', dfOneHotEncoder())])


combined_features = PandasFeatureUnion([("num", Npipe_prevapp),
                                        ("cat", Cpipe_prevapp)])


In [0]:
#train pipeline
final = combineSuppoutFiles()
train = pd.read_csv(r'/content/drive/My Drive/HCDR Project/application_train.csv')
Y = train['TARGET']

train, test, Y, test_y = train_test_split(train, Y, test_size=0.33, random_state=42,stratify=Y)



train_ids = train['SK_ID_CURR']
train.drop(['TARGET','SK_ID_CURR'], inplace = True, axis = 1)

train = combined_features.fit_transform(train)

train['SK_ID_CURR'] = train_ids

#merging app prev 
train = train.merge(final, how  = 'left', on = 'SK_ID_CURR')
train.drop(['SK_ID_CURR'], axis = 1, inplace = True)
train.fillna(0, inplace = True)


In [0]:
#test pipeline
#test = pd.read_csv(r'/content/drive/My Drive/HCDR Project/application_test.csv')
#test_y
test_ids = test['SK_ID_CURR']
test.drop(['SK_ID_CURR','TARGET'], inplace = True, axis = 1)
test = combined_features.transform(test)
test['SK_ID_CURR'] = test_ids
test = test.merge(final, how  = 'left', on = 'SK_ID_CURR')
test.drop(['SK_ID_CURR'], axis = 1, inplace = True)
test.fillna(0, inplace = True)
test.shape

(101479, 682)

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

#LinearSVC(C=0.01, penalty="l1", dual=False)

#finalPipe = Pipeline([('fs', SelectFromModel(RandomForestClassifier(n_estimators = 10, n_jobs = -1)))
#                     ,('mdl',GradientBoostingClassifier(class_weight = 'balanced') )])

finalPipe = Pipeline([('mdl',GradientBoostingClassifier(n_iter_no_change= 5,
                                                        validation_fraction = .10,
                                                       max_features = 'auto',
                                                       random_state = 42,
                                                       max_depth = 8,
                                                       verbose = 1
                                                       ))])


mdl = finalPipe.fit(train,Y)





      Iter       Train Loss   Remaining Time 
         1           0.5466          163.98m
         2           0.5353          163.23m
         3           0.5259          162.51m
         4           0.5189          158.95m
         5           0.5122          157.95m
         6           0.5062          154.79m


In [0]:
#roc(mdl.predict(test),test_y)
#submission(mdl, final, combined_features)
#import json
#Setupkaggle()
#!kaggle competitions submit -c home-credit-default-risk -f submission.csv -m "GBM"
#!kaggle competitions submissions home-credit-default-risk  

In [0]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K
from sklearn.utils import class_weight

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

Using TensorFlow backend.


In [0]:
#loss  = categorical_crossentropy,binary_crossentropy
#optimizer = rmsprop, sgd, adam, 
#activation = softmax, relu , sigmoid

weights = class_weight.compute_class_weight('balanced',
                                           np.unique(Y),
                                           Y)
model = tf.keras.models.Sequential([
    # tf.keras.layers.Dense(500,  activation='relu')
    #,tf.keras.layers.Dropout(.20)
   # tf.keras.layers.Dense(100,  activation='relu')
   # ,tf.keras.layers.Dropout(.60)
   # ,tf.keras.layers.Dense(80,  activation='relu')
   # ,tf.keras.layers.Dropout(.50)   
    #tf.keras.layers.Dense(64,  activation='relu')
    #,tf.keras.layers.Dropout(.30)
    tf.keras.layers.Dense(32,  activation='relu')
    ,tf.keras.layers.Dropout(.30)
    ,tf.keras.layers.Dense(16,  activation='relu')
    ,tf.keras.layers.Dropout(.30)
    ,tf.keras.layers.Dense(1,  activation='sigmoid')
])

model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['accuracy',sensitivity,specificity])

X_train, X_val, Y_train, Y_val = train_test_split(trn, Y, test_size=0.10, random_state=42,stratify=Y)
model.fit(np.array(X_train) ,np.array(Y_train)
          ,batch_size=2000
          ,epochs=10
          ,validation_data = (np.array(X_val),np.array(Y_val))
          ,callbacks=[es], class_weight=weights)


Train on 206032 samples, validate on 20604 samples
Epoch 1/10
206032/206032 [==============================] - 8s 41us/sample - loss: 0.4025 - acc: 0.8614 - sensitivity: 0.0705 - specificity: 0.9314 - val_loss: 0.2653 - val_acc: 0.9192 - val_sensitivity: 0.0000e+00 - val_specificity: 0.9999
Epoch 2/10
206032/206032 [==============================] - 8s 38us/sample - loss: 0.2962 - acc: 0.9180 - sensitivity: 0.0019 - specificity: 0.9985 - val_loss: 0.2557 - val_acc: 0.9193 - val_sensitivity: 0.0000e+00 - val_specificity: 1.0000
Epoch 3/10
206032/206032 [==============================] - 8s 38us/sample - loss: 0.2844 - acc: 0.9189 - sensitivity: 7.5393e-04 - specificity: 0.9996 - val_loss: 0.2550 - val_acc: 0.9193 - val_sensitivity: 0.0000e+00 - val_specificity: 1.0000
Epoch 4/10
206032/206032 [==============================] - 8s 38us/sample - loss: 0.2795 - acc: 0.9191 - sensitivity: 6.1909e-04 - specificity: 0.9997 - val_loss: 0.2533 - val_acc: 0.9193 - val_sensitivity: 0.0000e+00 - v

In [0]:
#import json
#Setupkaggle()

#Kaggle Setup (make sure kaggle.json is in your google drive home dir)
#!pip install kaggle
#Setupkaggle():
#!kaggle competitions files home-credit-default-risk
#!kaggle competitions submit -c home-credit-default-risk -f submission.csv -m "Logistic Weighted"
#!kaggle competitions submissions home-credit-default-risk